In [1]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [2]:
landmark_titles = [mp_pose.PoseLandmark(i).name for i in range(len(mp_pose.PoseLandmark))]        
full=[]
for land in landmark_titles:
    full.append(land+"_x")
    full.append(land+"_y")
    # full.append(land+"_z")
    full.append(land+"_visi")
print(full)

['NOSE_x', 'NOSE_y', 'NOSE_visi', 'LEFT_EYE_INNER_x', 'LEFT_EYE_INNER_y', 'LEFT_EYE_INNER_visi', 'LEFT_EYE_x', 'LEFT_EYE_y', 'LEFT_EYE_visi', 'LEFT_EYE_OUTER_x', 'LEFT_EYE_OUTER_y', 'LEFT_EYE_OUTER_visi', 'RIGHT_EYE_INNER_x', 'RIGHT_EYE_INNER_y', 'RIGHT_EYE_INNER_visi', 'RIGHT_EYE_x', 'RIGHT_EYE_y', 'RIGHT_EYE_visi', 'RIGHT_EYE_OUTER_x', 'RIGHT_EYE_OUTER_y', 'RIGHT_EYE_OUTER_visi', 'LEFT_EAR_x', 'LEFT_EAR_y', 'LEFT_EAR_visi', 'RIGHT_EAR_x', 'RIGHT_EAR_y', 'RIGHT_EAR_visi', 'MOUTH_LEFT_x', 'MOUTH_LEFT_y', 'MOUTH_LEFT_visi', 'MOUTH_RIGHT_x', 'MOUTH_RIGHT_y', 'MOUTH_RIGHT_visi', 'LEFT_SHOULDER_x', 'LEFT_SHOULDER_y', 'LEFT_SHOULDER_visi', 'RIGHT_SHOULDER_x', 'RIGHT_SHOULDER_y', 'RIGHT_SHOULDER_visi', 'LEFT_ELBOW_x', 'LEFT_ELBOW_y', 'LEFT_ELBOW_visi', 'RIGHT_ELBOW_x', 'RIGHT_ELBOW_y', 'RIGHT_ELBOW_visi', 'LEFT_WRIST_x', 'LEFT_WRIST_y', 'LEFT_WRIST_visi', 'RIGHT_WRIST_x', 'RIGHT_WRIST_y', 'RIGHT_WRIST_visi', 'LEFT_PINKY_x', 'LEFT_PINKY_y', 'LEFT_PINKY_visi', 'RIGHT_PINKY_x', 'RIGHT_PINKY_y',

In [3]:
with open('exercise_landmarks.csv', 'a', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Exercise'] + [f"{land}_{dim}" for land in landmark_titles for dim in ["x", "y", "visi"]])

In [3]:
cap = cv2.VideoCapture('./Videos/3.mp4')

In [4]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
# cap = cv2.VideoCapture(0)
exercise_name = input("Enter the exercise you are going to do: ")
recording = False
landmark_data = []

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame=cv2.resize(frame,(400,500))
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detectionsq
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )               
        try:
            landmarks = results.pose_landmarks.landmark
            if recording:
                # print(list(np.array([[landmark.x, landmark.y, landmark.z,landmark.visibility] for landmark in landmarks]).flatten()))
                landmark_data.append([exercise_name]  + list(np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in landmarks]).flatten()))
        except:
            pass
        
        cv2.imshow('Feed', image)
        key = cv2.waitKey(10)
        if key & 0xFF == ord('q'):
            break
        elif key & 0xFF == ord('r'):
            if not recording:
                print("Recording started.")
                recording = True
            else:
                print("Recording stopped.")
                recording = False
                
with open('exercise_landmarks.csv', 'a', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerows(landmark_data)
cap.release()
cv2.destroyAllWindows()


In [3]:
df=pd.read_csv('exercise_landmarks.csv')
df

,Exercise,NOSE_x,NOSE_y,NOSE_visi,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_visi,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_visi,...,LEFT_HEEL_visi,RIGHT_HEEL_x,RIGHT_HEEL_y,RIGHT_HEEL_visi,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_visi,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_visi
0,squat,0.505166,0.266312,0.982306,0.493333,0.280616,0.984479,0.485229,0.279498,0.984173,...,0.187821,0.547483,0.389793,0.174027,0.518469,0.544447,0.200514,0.554678,0.597746,0.205054
1,squat,0.488600,0.266425,0.983550,0.479235,0.258106,0.984903,0.472551,0.253804,0.984846,...,0.231121,0.536491,0.505046,0.232987,0.517602,0.627873,0.246120,0.564084,0.634770,0.248031
2,squat,0.468427,0.242144,0.985168,0.459510,0.257077,0.986381,0.453892,0.255100,0.986323,...,0.265076,0.500880,0.414890,0.264477,0.482435,0.455931,0.276498,0.505261,0.498170,0.273053
3,squat,0.478547,0.212118,0.986639,0.471765,0.243858,0.987729,0.466748,0.244324,0.987676,...,0.289519,0.500345,0.524527,0.280461,0.490410,0.635299,0.310126,0.502840,0.656740,0.296193
4,squat,0.472660,0.237043,0.987909,0.467909,0.267013,0.988887,0.464449,0.266849,0.988829,...,0.284305,0.464647,0.425869,0.281728,0.477803,0.532830,0.309488,0.467760,0.543756,0.297226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2031,Biceps-Pullup,0.393660,0.180372,0.998720,0.398628,0.150604,0.998779,0.404803,0.146812,0.998968,...,0.032902,0.637320,1.217673,0.028856,0.702907,1.285725,0.011736,0.560915,1.277939,0.010804
2032,Biceps-Pullup,0.389932,0.180420,0.998799,0.395981,0.151049,0.998859,0.402736,0.147291,0.999034,...,0.031803,0.638999,1.226297,0.027901,0.702552,1.297464,0.011201,0.560476,1.285897,0.010346
2033,Biceps-Pullup,0.389090,0.181329,0.998891,0.395103,0.152375,0.998948,0.402377,0.148411,0.999109,...,0.032096,0.653779,1.223555,0.028180,0.701623,1.297487,0.011121,0.566169,1.283148,0.010306
2034,Biceps-Pullup,0.389352,0.180296,0.998971,0.395605,0.151213,0.999025,0.402980,0.147207,0.999172,...,0.032955,0.658292,1.221481,0.028753,0.698965,1.295461,0.011218,0.569421,1.281101,0.010377


In [6]:
x=df.iloc[0:,1:]
y=df['Exercise']

In [7]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=10)

In [8]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1527, 99)
(509, 99)
(1527,)
(509,)


In [14]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
}

In [15]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [16]:
fit_models['rf'].predict(x_test)

array(['Biceps-Pullup', 'squat', 'Biceps-Pullup', 'Shoulder_Pulldown',
       'Shoulder_Pulldown', 'Biceps-Pullup', 'Biceps-Pullup',
       'Shoulder_Pulldown', 'Shoulder_Pulldown', 'squat', 'Biceps-Pullup',
       'squat', 'Shoulder_Pulldown', 'Biceps-Pullup', 'Shoulder_Pulldown',
       'squat', 'Biceps-Pullup', 'Biceps-Pullup', 'squat', 'squat',
       'Biceps-Pullup', 'Shoulder_Pulldown', 'squat', 'Biceps-Pullup',
       'squat', 'squat', 'Biceps-Pullup', 'Biceps-Pullup', 'squat',
       'squat', 'squat', 'squat', 'Biceps-Pullup', 'Shoulder_Pulldown',
       'Biceps-Pullup', 'squat', 'Biceps-Pullup', 'squat',
       'Shoulder_Pulldown', 'squat', 'squat', 'Biceps-Pullup',
       'Shoulder_Pulldown', 'Shoulder_Pulldown', 'Shoulder_Pulldown',
       'Biceps-Pullup', 'Shoulder_Pulldown', 'squat', 'Shoulder_Pulldown',
       'Biceps-Pullup', 'Biceps-Pullup', 'squat', 'squat',
       'Biceps-Pullup', 'squat', 'Shoulder_Pulldown', 'Biceps-Pullup',
       'squat', 'Biceps-Pullup', 'squat',

In [18]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

In [24]:
df=pd.DataFrame(x_test.iloc[0])
df
# df=df.transpose()
# model.predict(df)

,669
NOSE_x,0.461458
NOSE_y,0.304505
NOSE_visi,0.991804
LEFT_EYE_INNER_x,0.477028
LEFT_EYE_INNER_y,0.276571
...,...
LEFT_FOOT_INDEX_y,1.600600
LEFT_FOOT_INDEX_visi,0.012305
RIGHT_FOOT_INDEX_x,0.392259
RIGHT_FOOT_INDEX_y,1.620963


In [20]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [115]:
cap = cv2.VideoCapture(0)

In [42]:
cap = cv2.VideoCapture('./Videos/3.mp4')

In [54]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
cap = cv2.VideoCapture('./Videos/3.mp4')
recording = False
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame=cv2.resize(frame,(400,500))
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detectionsq
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )               
        try:
            landmarks = results.pose_landmarks.landmark
            if recording:
                data = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in landmarks]).flatten()
                df=pd.DataFrame([data],columns=full)
                predictions = model.predict(df)
                prob=model.predict_proba(df)
                # print(predictions[0])
                # print(prob[np.argmax(prob)])
                prob=prob.flatten()
                
                 # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, predictions[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                # # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(prob[np.argmax(prob)])
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"Error: {e}")
            # pass
        
        cv2.imshow('Mediapipe Feed', image)
        key = cv2.waitKey(10)
        if key & 0xFF == ord('q'):
            break
        elif key & 0xFF == ord('r'):
            if not recording:
                print("Recording started.")
                recording = True
            else:
                print("Recording stopped.")
                recording = False
cap.release()
cv2.destroyAllWindows()

Recording started.
Recording stopped.
